In [18]:
from AlgorithmicTrading.strategies.perceptron import Strategy
from AlgorithmicTrading.account import AccountBacktest, AccountLive
from AlgorithmicTrading.models.metatrader import ENUM_ACCOUNT_MARGIN_MODE, ENUM_TIMEFRAME
from AlgorithmicTrading.rates import Rates
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import MetaTrader5 as mt5
import pandas as pd

In [19]:
# Netting account
account = AccountLive.login(login=5013526569, server="MetaQuotes-Demo", password="h3xhhvuo")
# Backtest account
account = AccountBacktest.login(balance=100_000, margin_mode=ENUM_ACCOUNT_MARGIN_MODE.ACCOUNT_MARGIN_MODE_RETAIL_NETTING)
account

[INFO]: Successfull login to #5013526569 account
[INFO]: Successfull backtest account create


MqlAccountInfo(login=9999999, trade_mode=<ENUM_ACCOUNT_TRADE_MODE.ACCOUNT_TRADE_MODE_DEMO: 0>, leverage=100, limit_orders=200, margin_so_mode=<ENUM_ACCOUNT_STOPOUT_MODE.ACCOUNT_STOPOUT_MODE_PERCENT: 0>, trade_allowed=True, trade_expert=True, margin_mode=<ENUM_ACCOUNT_MARGIN_MODE.ACCOUNT_MARGIN_MODE_RETAIL_NETTING: 0>, currency_digits=2, fifo_close=False, balance=100000.0, credit=0.0, profit=0.0, equity=100000.0, margin=0.0, margin_free=100000.0, margin_level=0.0, margin_so_call=50.0, margin_so_so=30.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Backtest Name', server='Backtest Server', currency='USD', company='Backtest Company', orders=[], positions=[], history_deals=[], is_backtest_account=True)

In [20]:
n_rates = Rates.get_last_n_candles("USDJPY", timeframe=ENUM_TIMEFRAME.TIMEFRAME_M5).reset_index()
n_rates

,time,open,high,low,close,tick_volume
0,2023-06-05 03:40:00+00:00,140.206,140.232,140.184,140.189,347
1,2023-06-05 03:45:00+00:00,140.188,140.189,140.148,140.155,304
2,2023-06-05 03:50:00+00:00,140.155,140.183,140.123,140.127,475
3,2023-06-05 03:55:00+00:00,140.126,140.173,140.099,140.113,507
4,2023-06-05 04:00:00+00:00,140.113,140.119,140.040,140.045,575
...,...,...,...,...,...,...
9995,2023-07-21 23:35:00+00:00,141.755,141.776,141.755,141.776,82
9996,2023-07-21 23:40:00+00:00,141.776,141.800,141.775,141.789,62
9997,2023-07-21 23:45:00+00:00,141.789,141.805,141.780,141.805,68
9998,2023-07-21 23:50:00+00:00,141.805,141.837,141.792,141.834,63


In [21]:
data = Strategy.features_engineering(n_rates).dropna()
data

,open,high,low,close,tick_volume,Target - returns,Feature - 5 SMA,Feature - 10 SMA,Feature - 20 SMA,Feature - 30 SMA,...,Feature - 21 RSI SMA,Feature - MACD Line,Feature - MACD Signal Line,Feature - MACD Diff,Feature - Bollinger Bands High Band,Feature - Bollinger Bands Mid Band,Feature - Bollinger Bands Low Band,Feature - Bollinger Bands P Band,Feature - Bollinger Bands W Band,Feature - Acc/Dist Index
33,140.109,140.114,140.103,140.112,242,-0.025,140.109943,140.118276,140.124714,140.125296,...,47.356218,-0.009020,-0.002551,-0.006469,140.221197,140.14010,140.059003,0.326751,0.115737,-1556.221135
34,140.116,140.116,140.083,140.091,329,-0.033,140.103629,140.113313,140.121403,140.122846,...,46.761627,-0.010738,-0.004188,-0.006549,140.219692,140.13655,140.053408,0.226071,0.118659,-1725.705984
35,140.090,140.091,140.035,140.057,343,-0.044,140.088086,140.103066,140.115098,140.118174,...,45.670281,-0.014673,-0.006285,-0.008388,140.222946,140.13285,140.042754,0.079061,0.128587,-1799.205984
36,140.057,140.058,139.997,140.013,261,0.041,140.063057,140.086681,140.105129,140.110760,...,44.112041,-0.021099,-0.009248,-0.011851,140.231226,140.12705,140.022874,-0.047392,0.148688,-1923.287951
37,140.013,140.056,139.990,140.054,285,-0.006,140.060038,140.080736,140.100148,140.106783,...,43.721425,-0.022623,-0.011923,-0.010700,140.232389,140.12345,140.014511,0.181245,0.155491,-1655.560678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,141.718,141.760,141.714,141.755,49,0.021,141.735349,141.738583,141.741400,141.736891,...,51.955891,0.000740,0.005872,-0.005132,141.798444,141.75230,141.706156,0.529256,0.065105,82194.531498
9995,141.755,141.776,141.755,141.776,82,0.013,141.748899,141.745386,141.744695,141.739414,...,52.412093,0.003560,0.005410,-0.001850,141.790835,141.75070,141.710565,0.815184,0.056628,82276.531498
9996,141.776,141.800,141.775,141.789,62,0.016,141.762266,141.753316,141.748915,141.742613,...,52.985657,0.006766,0.005681,0.001085,141.790591,141.75065,141.710709,0.980079,0.056354,82283.971498
9997,141.789,141.805,141.780,141.805,68,0.029,141.776511,141.762713,141.754256,141.746638,...,53.697800,0.010477,0.006640,0.003837,141.799472,141.75305,141.706628,1.059542,0.065497,82351.971498


In [22]:
X = data.drop(columns=["Target - returns"]).values


p20 = data["Target - returns"].quantile(.2) 
p80 = data["Target - returns"].quantile(.8)
y = (data["Target - returns"] < p20) | (data["Target - returns"] > p80).values
# y = (data["Target - returns"] > 0).values
y

array([False, False, False, ...,  True,  True, False])

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
model = MLPRegressor(hidden_layer_sizes=[30, 20], random_state=42)
pipeline = make_pipeline(StandardScaler(), model)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
y_pred = (y_pred >= 0.5)

# mean_squared_error(y_pred, y_test, squared=False)
print(str(round(accuracy_score(y_pred=y_pred, y_true=y_test) * 100)) + "%")

50%


C:\Users\pedro\.pyenv\pyenv-win\versions\3.11.3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
